In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
df = pd.read_spss('ougd_2018_work.sav')

In [3]:
df.rename(columns={'rik_fa_1': 'year',
                   'code_fam': 'household_code',
                   'w_q': 'statistical_weight',
                   'tp_ns_p': 'settlement_type',
                   'cod_obl': 'region',
                   'hsize': 'number_of_people',
                   'type_dom': 'household_type',
                   'h_ch': 'number_of_children',
                   'h_elder': 'number_of_incapacitated',
                   'house': 'house_type',
                   'stotal': 'house_area', 
                   'heating': 'central_heating', 
                   'heatind': 'individual_heating',
                   'h045': 'energy_costs',
                   'Q_refr': 'refrigerator',
                   'Q_froz': 'freezer',
                   'Q_wash': 'washing_machine',
                   'Q_vacum': 'vacuum_cleaner',
                   'Q_tv_cl': 'colour_TV',
                   'Q_mcwav': 'microwave',
                   'Q_combi': 'food_processor',
                   'Q_condi': 'air_conditioning',
                   'Q_crocer': 'dishwasher',
                   'Q_compu': 'PC',
                   'Q_noutb': 'laptop',
                   'Q_iron': 'iron',
                   'gaskol': 'gas_station',
                   'gascentr': 'centralized_gas',
                   'gasbal': 'bottled_gas',
                   'elektrpl': 'electric_stove'
                    }, 
          inplace=True)

In [4]:
change_household_type = {"домогосподарство з дiтьми":1, 
                         "домогосподарство без дiтей":2}

change_house_type = {"окрема квартира":1, 
                    "індивідуальний будинок":2,
                    "частина індивідуального будинку":3,
                    "інше":4}

change_house_area = {"до 40 м кв":1, 
                    "40,01 - 60,0 м кв":2,
                    "60,01 - 80,0 м кв":3,
                    "понад 80,01 м кв":4}

change_central_heating = {"так":1, 
                         "ні":2,
                         "не знає, відмова від відповіді":9}

change_individual_heating = {"так":1, 
                             "ні":2, 
                             "не знає, відмова від відповіді":9}

change_settlement_type = {"Велике місто":1, 
                          "Мале місто":2, 
                          "Сільська місцевість":9}

change_electrical_appliances = {"ні":0, 
                                "так":1}

change_gas = {"так":1, 
              "ні":2, 
              "не знає, відмова від відповіді":9}

In [5]:
household_table = df[["household_type", "number_of_people" , "number_of_incapacitated"]].copy(deep = True)

household_table = household_table.replace({'household_type': change_household_type})

household_table.fillna(value=0, inplace = True)

household_table["retirees"] = ((household_table.number_of_people == household_table.number_of_incapacitated) & \
                              (household_table.household_type == 2)).astype(int)
household_table.loc[household_table.retirees == 1,'household_type'] = 3

household_table = household_table.drop(labels=['retirees', 'number_of_people', 'number_of_incapacitated'], axis=1)

In [6]:
heating_table = df[["central_heating", "individual_heating", "settlement_type", "house_type"]].copy(deep = True)

heating_table = heating_table.replace({'central_heating': change_central_heating,
                                       'individual_heating': change_individual_heating,
                                       'settlement_type': change_settlement_type,
                                       'house_type': change_house_type})

heating_table.fillna(value=0, inplace = True)

heating_table["heating"] = ((heating_table.central_heating == 2) & (heating_table.individual_heating == 2) & \
                            ((heating_table.house_type ==2) | (heating_table.house_type ==3) | \
                            ((heating_table.house_type == 4) & \
                            ((heating_table.settlement_type == 9) | (heating_table.settlement_type == 2))))).astype(int)

heating_table.loc[heating_table.heating == 1,'heating'] = 2
heating_table.loc[heating_table.heating == 0,'heating'] = 1

heating_table.loc[heating_table.central_heating == 1,'heating'] = 1
heating_table.loc[heating_table.individual_heating == 1,'heating'] = 2

heating_table = heating_table.drop(labels=['central_heating', 'individual_heating', 'settlement_type', 'house_type'], axis=1)

In [7]:
house_type_table = df[["house_type"]].copy(deep = True)

house_type_table = house_type_table.replace({'house_type': change_house_type})

house_type_table.fillna(value=0, inplace = True)

house_type_table.loc[house_type_table.house_type == 1,'house'] = 1
house_type_table.loc[(house_type_table.house_type == 2) | (house_type_table.house_type == 3),'house'] = 2
house_type_table.loc[house_type_table.house_type == 4,'house'] = 3

house_type_table = house_type_table.drop(labels='house_type', axis=1)

In [8]:
area_table = df[["house_area"]].copy(deep = True)

area_table = area_table.replace({'house_area': change_house_area})

area_table.fillna(value=0, inplace = True)

In [9]:
settlement_table = df[["settlement_type"]].copy(deep = True)

settlement_table = settlement_table.replace({'settlement_type': change_settlement_type})

settlement_table.fillna(value=0, inplace = True)

settlement_table.loc[(settlement_table.settlement_type == 1) | (settlement_table.settlement_type == 2),'settlement'] = 1
settlement_table.loc[settlement_table.settlement_type == 9,'settlement'] = 2

settlement_table = settlement_table.drop(labels='settlement_type', axis=1)

In [10]:
people_table = df[["statistical_weight", "number_of_people"]].copy(deep = True)

people_table["total_number_of_people"] = people_table['statistical_weight'] * people_table['number_of_people']

people_table = people_table.drop(labels=['statistical_weight', 'number_of_people'], axis=1)

In [11]:
energy_costs_table = df[["statistical_weight", "energy_costs"]].copy(deep = True)

energy_costs_table["total_energy_costs"] = energy_costs_table['statistical_weight'] * energy_costs_table['energy_costs']

energy_costs_table = energy_costs_table.drop(labels=['statistical_weight', 'energy_costs'], axis=1)

In [12]:
electrical_appliances_table = df[["statistical_weight", "refrigerator", "freezer", 
                                  "washing_machine", "vacuum_cleaner", "colour_TV", 
                                  "microwave", "food_processor", "air_conditioning", 
                                  "dishwasher", "PC", "laptop", "iron", 
                                  "electric_stove", "gas_station"]].copy(deep = True)

electrical_appliances_table = electrical_appliances_table.replace({'refrigerator': change_electrical_appliances,
                                                                   'freezer': change_electrical_appliances,
                                                                   'washing_machine': change_electrical_appliances,
                                                                   'vacuum_cleaner': change_electrical_appliances,
                                                                   'colour_TV': change_electrical_appliances,
                                                                   'microwave': change_electrical_appliances,
                                                                   'food_processor': change_electrical_appliances,
                                                                   'air_conditioning': change_electrical_appliances,
                                                                   'dishwasher': change_electrical_appliances,
                                                                   'PC': change_electrical_appliances,
                                                                   'laptop': change_electrical_appliances,
                                                                   'iron': change_electrical_appliances,
                                                                   'electric_stove': change_gas,
                                                                   'gas_station': change_gas})

electrical_appliances_table.loc[(electrical_appliances_table.electric_stove == 2) | \
                                (electrical_appliances_table.electric_stove == 9),'electric_stove'] = 0
electrical_appliances_table.loc[electrical_appliances_table.electric_stove == 1,'electric_stove'] = 1

electrical_appliances_table.loc[(electrical_appliances_table.gas_station == 2) | \
                                (electrical_appliances_table.gas_station == 9),'gas_station'] = 0
electrical_appliances_table.loc[electrical_appliances_table.gas_station == 1,'gas_station'] = 1


electrical_appliances_table["total_gas_station"] = electrical_appliances_table['statistical_weight'] * \
                                                    electrical_appliances_table['gas_station']
electrical_appliances_table["total_electric_stove"] = electrical_appliances_table['statistical_weight'] * \
                                                    electrical_appliances_table['electric_stove']
electrical_appliances_table["total_refrigerator"] = electrical_appliances_table['statistical_weight'] * \
                                                    electrical_appliances_table['refrigerator']
electrical_appliances_table["total_freezer"] = electrical_appliances_table['statistical_weight'] * \
                                               electrical_appliances_table['freezer']
electrical_appliances_table["total_washing_machine"] = electrical_appliances_table['statistical_weight'] * \
                                                       electrical_appliances_table['washing_machine']
electrical_appliances_table["total_vacuum_cleaner"] = electrical_appliances_table['statistical_weight'] * \
                                                      electrical_appliances_table['vacuum_cleaner']
electrical_appliances_table["total_colour_TV"] = electrical_appliances_table['statistical_weight'] * \
                                                 electrical_appliances_table['colour_TV']
electrical_appliances_table["total_microwave"] = electrical_appliances_table['statistical_weight'] * \
                                                 electrical_appliances_table['microwave']
electrical_appliances_table["total_food_processor"] = electrical_appliances_table['statistical_weight'] * \
                                                      electrical_appliances_table['food_processor']
electrical_appliances_table["total_air_conditioning"] = electrical_appliances_table['statistical_weight'] * \
                                                        electrical_appliances_table['air_conditioning']
electrical_appliances_table["total_dishwasher"] = electrical_appliances_table['statistical_weight'] * \
                                                  electrical_appliances_table['dishwasher']
electrical_appliances_table["total_PC"] = electrical_appliances_table['statistical_weight'] * \
                                          electrical_appliances_table['PC']
electrical_appliances_table["total_laptop"] = electrical_appliances_table['statistical_weight'] * \
                                              electrical_appliances_table['laptop']
electrical_appliances_table["total_iron"] = electrical_appliances_table['statistical_weight'] * \
                                            electrical_appliances_table['iron']

electrical_appliances_table = electrical_appliances_table.drop(labels=['statistical_weight', 'refrigerator', 'freezer', 
                                                                       'washing_machine', 'vacuum_cleaner', 'colour_TV', 
                                                                       'microwave', 'food_processor', 'air_conditioning', 
                                                                       'dishwasher', 'PC', 'laptop', 'iron',
                                                                       'electric_stove', 'gas_station'], axis=1)

In [13]:
energy_table_0 = df[["statistical_weight", "centralized_gas", "bottled_gas"]].copy(deep = True)

energy_table_0 = energy_table_0.replace({'centralized_gas': change_gas,
                                         'bottled_gas': change_gas})

In [14]:
energy_table = pd.concat([energy_table_0.astype(int), house_type_table.astype(int), 
                          heating_table, settlement_table.astype(int)], axis = 1)

In [15]:
energy_table.loc[(energy_table.heating == 1) | (energy_table.heating == 2),'electric_power'] = 1

energy_table.loc[energy_table.heating == 1,'thermal_energy'] = 1
energy_table.loc[energy_table.heating == 2,'thermal_energy'] = 0

energy_table.loc[energy_table.centralized_gas == 1,'natural_gas'] = 1
energy_table.loc[energy_table.centralized_gas == 2,'natural_gas'] = 0

energy_table.loc[(energy_table.house == 2),'solid_fuel'] = 1
energy_table.loc[(energy_table.house == 1) | (energy_table.house == 3),'solid_fuel'] = 0

energy_table.loc[(energy_table.bottled_gas == 1),'liquefied_gas'] = 1 
energy_table.loc[(energy_table.bottled_gas == 2),'liquefied_gas'] = 0

energy_table.loc[(energy_table.house == 1) | (energy_table.house == 3),'biofuels'] = 0
energy_table.loc[(energy_table.house == 2) & (energy_table.settlement == 1),'biofuels'] = 0
energy_table.loc[(energy_table.house == 2) & (energy_table.settlement == 2),'biofuels'] = 1

In [16]:
energy_table["total_electric_power"] = energy_table['statistical_weight'] * energy_table['electric_power']
energy_table["total_thermal_energy"] = energy_table['statistical_weight'] * energy_table['thermal_energy']
energy_table["total_natural_gas"] = energy_table['statistical_weight'] * energy_table['natural_gas']
energy_table["total_solid_fuel"] = energy_table['statistical_weight'] * energy_table['solid_fuel']
energy_table["total_liquefied_gas"] = energy_table['statistical_weight'] * energy_table['liquefied_gas']
energy_table["total_biofuels"] = energy_table['statistical_weight'] * energy_table['biofuels']

In [17]:
energy_table = energy_table.drop(labels=['centralized_gas', 'bottled_gas', 'house', 'heating', 'settlement',
                                         'electric_power', 'thermal_energy', 'natural_gas', 'solid_fuel', 
                                         'liquefied_gas', 'biofuels', 'statistical_weight'], axis=1)

In [18]:
category_first = pd.concat([household_table, house_type_table.astype(int), area_table.astype(int), 
                            heating_table, settlement_table.astype(int), people_table.astype(int),
                            energy_costs_table.astype(float), electrical_appliances_table.astype(float),
                            energy_table.astype(float)], axis = 1)

In [19]:
df.household_type = category_first.household_type
df.settlement_type = category_first.settlement
df.house_type = category_first.house
df.house_area = category_first.house_area

df = df.drop(labels=['central_heating', 'individual_heating', 'refrigerator', 
                     'freezer', 'washing_machine', 'vacuum_cleaner', 
                     'colour_TV', 'microwave', 'food_processor', 
                     'air_conditioning', 'dishwasher', 'PC', 
                     'laptop', 'iron', 'electric_stove'], axis=1)

df_1 = pd.concat([df, heating_table, people_table.astype(int), energy_costs_table, 
                  electrical_appliances_table, energy_table.astype(float)], axis = 1)

In [20]:
category_second = df_1.groupby(['household_type', 'house_type', 
                                'house_area', 'heating', 
                                'settlement_type'])['statistical_weight', 'total_number_of_people', 
                                                    'total_energy_costs', 'total_refrigerator', 
                                                    'total_freezer', 'total_washing_machine', 
                                                    'total_vacuum_cleaner', 'total_colour_TV', 
                                                    'total_microwave', 'total_food_processor', 
                                                    'total_air_conditioning', 'total_dishwasher', 
                                                    'total_PC', 'total_laptop', 'total_iron',
                                                    'total_electric_stove', 'total_electric_power',
                                                    'total_thermal_energy', 'total_natural_gas',
                                                    'total_solid_fuel', 'total_liquefied_gas',
                                                    'total_biofuels', 'total_gas_station'].sum()

C:\Users\Vadim\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
number_of_households = df_1.groupby(['household_type', 'house_type', 
                                     'house_area', 'heating', 'settlement_type']).household_code.nunique().values

In [22]:
category_second["number_of_households"] = number_of_households

In [23]:
category_second["category"] = list(range(1,len(category_second) + 1))

In [24]:
new_category = category_second.reset_index().set_index('category')

In [25]:
new_category["smth"] = ((new_category.number_of_households <= 10) & (new_category.settlement_type == 1)).astype(int)

new_category.loc[new_category.smth == 1,'household_type'] = 11
new_category.loc[new_category.smth == 1,'house_type'] = 11
new_category.loc[new_category.smth == 1,'house_area'] = 11
new_category.loc[new_category.smth == 1,'heating'] = 11

new_category["smb"] = ((new_category.number_of_households <= 10) & (new_category.settlement_type == 2)).astype(int)

new_category.loc[new_category.smb == 1,'household_type'] = 22
new_category.loc[new_category.smb == 1,'house_type'] = 22
new_category.loc[new_category.smb == 1,'house_area'] = 22
new_category.loc[new_category.smb == 1,'heating'] = 22

new_category = new_category.drop(labels=['smth', 'smb'], axis=1)

In [26]:
# new_category

In [27]:
new_category_2 = new_category.groupby(['household_type', 'house_type', 
                                       'house_area', 'heating', 
                                       'settlement_type'])['statistical_weight', 'total_number_of_people', 
                                                           'total_energy_costs', 'total_refrigerator', 
                                                           'total_freezer', 'total_washing_machine', 
                                                           'total_vacuum_cleaner', 'total_colour_TV', 
                                                           'total_microwave', 'total_food_processor', 
                                                           'total_air_conditioning', 'total_dishwasher', 
                                                           'total_PC', 'total_laptop', 'total_iron',
                                                           'total_electric_stove', 'total_electric_power',
                                                           'total_thermal_energy', 'total_natural_gas',
                                                           'total_solid_fuel', 'total_liquefied_gas',
                                                           'total_biofuels', 'total_gas_station'].sum()

C:\Users\Vadim\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
new_category_2["category"] = list(range(1,len(new_category_2) + 1))

new_category_3 = new_category_2.reset_index().set_index('category')

In [29]:
new_category_3["total_energy_costs"] = new_category_3['total_energy_costs'] / new_category_3['statistical_weight']
new_category_3["total_refrigerator"] = new_category_3['total_refrigerator'] / new_category_3['statistical_weight']
new_category_3["total_freezer"] = new_category_3['total_freezer'] / new_category_3['statistical_weight']
new_category_3["total_washing_machine"] = new_category_3['total_washing_machine'] / new_category_3['statistical_weight']
new_category_3["total_vacuum_cleaner"] = new_category_3['total_vacuum_cleaner'] / new_category_3['statistical_weight']
new_category_3["total_colour_TV"] = new_category_3['total_colour_TV'] / new_category_3['statistical_weight']
new_category_3["total_microwave"] = new_category_3['total_microwave'] / new_category_3['statistical_weight']
new_category_3["total_food_processor"] = new_category_3['total_food_processor'] / new_category_3['statistical_weight']
new_category_3["total_air_conditioning"] = new_category_3['total_air_conditioning'] / new_category_3['statistical_weight']
new_category_3["total_dishwasher"] = new_category_3['total_dishwasher'] / new_category_3['statistical_weight']
new_category_3["total_PC"] = new_category_3['total_PC'] / new_category_3['statistical_weight']
new_category_3["total_laptop"] = new_category_3['total_laptop'] / new_category_3['statistical_weight']
new_category_3["total_iron"] = new_category_3['total_iron'] / new_category_3['statistical_weight']
new_category_3["total_electric_stove"] = new_category_3['total_electric_stove'] / new_category_3['statistical_weight']
new_category_3["total_electric_power"] = new_category_3['total_electric_power'] / new_category_3['statistical_weight']
new_category_3["total_thermal_energy"] = new_category_3['total_thermal_energy'] / new_category_3['statistical_weight']
new_category_3["total_natural_gas"] = new_category_3['total_natural_gas'] / new_category_3['statistical_weight']
new_category_3["total_solid_fuel"] = new_category_3['total_solid_fuel'] / new_category_3['statistical_weight']
new_category_3["total_liquefied_gas"] = new_category_3['total_liquefied_gas'] / new_category_3['statistical_weight']
new_category_3["total_biofuels"] = new_category_3['total_biofuels'] / new_category_3['statistical_weight']
new_category_3["total_gas_station"] = new_category_3['total_gas_station'] / new_category_3['statistical_weight']

In [30]:
# new_category_3 = new_category_3.drop(labels=['household_type', 'house_type', 
#                                              'house_area', 'heating', 'settlement_type'], axis=1)

In [31]:
# new_category_3

In [32]:
new_category_4 = new_category_3.round({'total_energy_costs':2, 'total_refrigerator':2,
                                       'total_freezer':2, 'total_washing_machine':2,
                                       'total_vacuum_cleaner':2, 'total_colour_TV':2,
                                       'total_microwave':2, 'total_food_processor':2,
                                       'total_air_conditioning':2, 'total_dishwasher':2,
                                       'total_PC':2, 'total_laptop':2,
                                       'total_iron':2, 'total_electric_stove':2,
                                       'total_electric_power':2, 'total_thermal_energy':2,
                                       'total_natural_gas':2, 'total_solid_fuel':2,
                                       'total_liquefied_gas':2, 'total_biofuels':2,
                                       'total_gas_station':2})

In [33]:
new_category_4.loc['Ukraine'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

new_category_4['cumsum_1'] = new_category_4['total_number_of_people'].cumsum()
new_category_4['cumsum_2'] = new_category_4['total_energy_costs'].cumsum()
new_category_4['cumsum_3'] = new_category_4['total_refrigerator'].cumsum()
new_category_4['cumsum_4'] = new_category_4['total_freezer'].cumsum()
new_category_4['cumsum_5'] = new_category_4['total_washing_machine'].cumsum()
new_category_4['cumsum_6'] = new_category_4['total_vacuum_cleaner'].cumsum()
new_category_4['cumsum_7'] = new_category_4['total_colour_TV'].cumsum()
new_category_4['cumsum_8'] = new_category_4['total_microwave'].cumsum()
new_category_4['cumsum_9'] = new_category_4['total_food_processor'].cumsum()
new_category_4['cumsum_10'] = new_category_4['total_air_conditioning'].cumsum()
new_category_4['cumsum_11'] = new_category_4['total_dishwasher'].cumsum()
new_category_4['cumsum_12'] = new_category_4['total_PC'].cumsum()
new_category_4['cumsum_13'] = new_category_4['total_laptop'].cumsum()
new_category_4['cumsum_14'] = new_category_4['total_iron'].cumsum()
new_category_4['cumsum_15'] = new_category_4['statistical_weight'].cumsum()
new_category_4['cumsum_16'] = new_category_4['total_electric_stove'].cumsum()
new_category_4['cumsum_17'] = new_category_4['total_electric_power'].cumsum()
new_category_4['cumsum_18'] = new_category_4['total_thermal_energy'].cumsum()
new_category_4['cumsum_19'] = new_category_4['total_natural_gas'].cumsum()
new_category_4['cumsum_20'] = new_category_4['total_solid_fuel'].cumsum()
new_category_4['cumsum_21'] = new_category_4['total_liquefied_gas'].cumsum()
new_category_4['cumsum_22'] = new_category_4['total_biofuels'].cumsum()
new_category_4['cumsum_23'] = new_category_4['total_gas_station'].cumsum()

In [34]:
# new_category_4.iloc[56]

In [35]:
new_category_4.loc[new_category_4.total_number_of_people == 0, 'total_number_of_people'] = 38549710

new_category_4.loc[new_category_4.statistical_weight == 0 , 'statistical_weight'] = 14934921

new_category_4.loc[new_category_4.total_energy_costs == 0, 'total_energy_costs'] = 449442.39

new_category_4.loc[(new_category_4.total_refrigerator == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_refrigerator'] = 55.07 / 56

new_category_4.loc[(new_category_4.total_freezer == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_freezer'] = 12.14 / 56

new_category_4.loc[(new_category_4.total_washing_machine == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_washing_machine'] = 50.23 / 56

new_category_4.loc[(new_category_4.total_vacuum_cleaner == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_vacuum_cleaner'] = 44.52 / 56

new_category_4.loc[(new_category_4.total_colour_TV == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_colour_TV'] = 53.36 / 56

new_category_4.loc[(new_category_4.total_microwave == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_microwave'] = 29.49 / 56

new_category_4.loc[(new_category_4.total_food_processor == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_food_processor'] = 9.38 / 56

new_category_4.loc[(new_category_4.total_air_conditioning == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_air_conditioning'] = 6.11 / 56

new_category_4.loc[(new_category_4.total_dishwasher == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_dishwasher'] = 0.87 / 56

new_category_4.loc[(new_category_4.total_PC == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_PC'] = 19.49 / 56

new_category_4.loc[(new_category_4.total_laptop == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_laptop'] = 16.86 / 56

new_category_4.loc[(new_category_4.total_iron == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_iron'] = 53.04 / 56

new_category_4.loc[(new_category_4.total_electric_stove == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_electric_stove'] = 3 / 56

new_category_4.loc[(new_category_4.total_electric_power == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_electric_power'] = 56 / 56

new_category_4.loc[(new_category_4.total_thermal_energy == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_thermal_energy'] = 16.13 / 56

new_category_4.loc[(new_category_4.total_natural_gas == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_natural_gas'] = 43.45 / 56

new_category_4.loc[(new_category_4.total_solid_fuel == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_solid_fuel'] = 24.16 / 56

new_category_4.loc[(new_category_4.total_liquefied_gas == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_liquefied_gas'] = 6.71 / 56

new_category_4.loc[(new_category_4.total_biofuels == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_biofuels'] = 12.09 / 56

new_category_4.loc[(new_category_4.total_gas_station == 0) & \
                   (new_category_4.total_number_of_people == 38549710), 'total_gas_station'] = 9.81 / 56

In [36]:
new_category_4 = new_category_4.drop(labels=['cumsum_1', 'cumsum_2', 'cumsum_3', 'cumsum_4', 'cumsum_5', 
                                             'cumsum_6', 'cumsum_7', 'cumsum_8', 'cumsum_9', 'cumsum_10',
                                             'cumsum_11', 'cumsum_12', 'cumsum_13', 'cumsum_14', 'cumsum_15',
                                             'cumsum_16', 'cumsum_17', 'cumsum_18', 'cumsum_19', 'cumsum_20',
                                             'cumsum_21', 'cumsum_22', 'cumsum_23'], axis=1)

In [37]:
new_category_4 = new_category_4.round({'total_energy_costs':2, 'total_refrigerator':2,
                                       'total_freezer':2, 'total_washing_machine':2,
                                       'total_vacuum_cleaner':2, 'total_colour_TV':2,
                                       'total_microwave':2, 'total_food_processor':2,
                                       'total_air_conditioning':2, 'total_dishwasher':2,
                                       'total_PC':2, 'total_laptop':2,
                                       'total_iron':2, 'total_electric_stove':2,
                                       'total_electric_power':2, 'total_thermal_energy':2,
                                       'total_natural_gas':2, 'total_solid_fuel':2,
                                       'total_liquefied_gas':2, 'total_biofuels':2,
                                       'total_gas_station': 2})

new_category_4[['statistical_weight']] = new_category_4[['statistical_weight']].astype(int)

new_category_4.rename(columns={'statistical_weight': 'total_number_of_households'}, inplace=True)

In [38]:
new_category_4 = new_category_4.drop(labels=['total_number_of_people'], axis=1)

In [39]:
new_category_4["energy_consumption_ttoe"] = [407, 62, 1234, 362, 46, 807, 235, 112, 48, 146, 
                                                56, 312, 503, 438, 478, 520, 982, 164, 365, 70, 
                                                1338, 23, 305, 18, 672, 128, 21, 97, 49, 111, 
                                                351, 403, 404, 637, 491, 591, 87, 116, 36, 533, 
                                                128, 9, 263, 80, 60, 17, 54, 62, 157, 358, 
                                                229, 357, 201, 225, 89, 160, 16203]

In [40]:
ALL_VALUES = set()

def get_distinct_values(max_value: int, min_value: int = 0.1):
  global ALL_VALUES
  value = np.random.uniform(low=min_value, high=max_value)
  while value in ALL_VALUES:
    value = np.random.uniform(low=min_value, high=max_value)
  ALL_VALUES.add(value)
  return value

In [41]:
def el_lighting_func(row):
    if (row['total_refrigerator'] == 0) & (row['total_freezer'] == 0) & (row['total_washing_machine'] == 0) & \
       (row['total_vacuum_cleaner'] == 0) & (row['total_colour_TV'] == 0) & (row['total_microwave'] == 0) & \
       (row['total_food_processor'] == 0) & (row['total_air_conditioning'] == 0) & (row['total_dishwasher'] == 0) & \
       (row['total_PC'] == 0) & (row['total_laptop'] == 0) & (row['total_iron'] == 0) & (row['total_electric_stove'] == 0):
        return 0
    else:
        return get_distinct_values(800, 5)
    
def el_water_heating_func(row):
    if (row['total_electric_stove'] == 0):
        return 0
    else:
        return get_distinct_values(170, 1)
    
def el_cooking_func(row):
    if (row['total_microwave'] == 0) & (row['total_food_processor'] == 0) & (row['total_electric_stove'] == 0):
        return 0
    else:
        return get_distinct_values(120, 1)

In [42]:
new_category_4.loc[:, 'el_heating'] = new_category_4.heating.apply(lambda x: 0 if x == 1 or x == 11 
                                                                   else get_distinct_values(80, 1))
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'el_heating'] = 163.823
ALL_VALUES.clear()

new_category_4.loc[:, 'el_conditioning'] = new_category_4.total_air_conditioning.apply(lambda x: 0 if x == 0 
                                                                                       else get_distinct_values(30, 1))
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'el_conditioning'] = 71.093
ALL_VALUES.clear()

new_category_4['el_water_heating'] = new_category_4.apply(el_water_heating_func, axis=1)
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'el_water_heating'] = 321.464
ALL_VALUES.clear()

new_category_4['el_cooking'] = new_category_4.apply(el_cooking_func, axis=1)
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'el_cooking'] = 262.735
ALL_VALUES.clear()

new_category_4['el_lighting'] = new_category_4.apply(el_lighting_func, axis=1)
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'el_lighting'] = 2265.703
ALL_VALUES.clear()


new_category_4.loc[:, 'el_other'] = new_category_4.settlement_type.apply(lambda x: 0 if x == 0 
                                                                         else get_distinct_values(3))
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'el_other'] = 6.182
ALL_VALUES.clear()

In [43]:
new_category_4.loc[:, 'th_heating'] = new_category_4.heating.apply(lambda x: 0 if x == 2 or x == 22 
                                                                   else get_distinct_values(700, 5))  
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'th_heating'] = 1614.681
ALL_VALUES.clear()

new_category_4.loc[(new_category_4.total_air_conditioning >= 0), 'th_conditioning'] = 0

new_category_4.loc[:, 'th_water_heating'] = new_category_4.heating.apply(lambda x: 0 if x == 2 or x == 22 
                                                                         else get_distinct_values(200, 2))
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'th_water_heating'] = 656.319
ALL_VALUES.clear()

new_category_4.loc[new_category_4.total_microwave >= 0, 'th_cooking'] = 0

new_category_4.loc[(new_category_4.total_PC >= 0), 'th_lighting'] = 0

new_category_4.loc[new_category_4.total_PC >= 0, 'th_other'] = 0

In [44]:
def gas_heating_func(row):
    if (row['heating'] == 1) | (row['heating'] == 11) | (row['total_natural_gas'] == 0):
        return 0
    else:
        return get_distinct_values(2000, 10)
    
def gas_water_heating_func(row):
    if (row['total_natural_gas'] == 0) & (row['total_gas_station'] == 0):
        return 0
    else:
        return get_distinct_values(300, 5)

In [45]:
new_category_4['gas_heating'] = new_category_4.apply(gas_heating_func, axis=1)   
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'gas_heating'] = 5083.065
ALL_VALUES.clear()

new_category_4.loc[(new_category_4.total_air_conditioning >= 0), 'gas_conditioning'] = 0

new_category_4['gas_water_heating'] = new_category_4.apply(gas_water_heating_func, axis=1)  
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'gas_water_heating'] = 1103.503
ALL_VALUES.clear()

new_category_4.loc[:, 'gas_cooking'] = new_category_4.total_natural_gas.apply(lambda x: 0 if x == 0
                                                                              else get_distinct_values(800, 5)) 
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'gas_cooking'] = 2502.432
ALL_VALUES.clear()

new_category_4.loc[new_category_4.total_PC >= 0, 'gas_lighting'] = 0

new_category_4.loc[new_category_4.total_PC >= 0, 'gas_other'] = 0

In [46]:
def solid_heating_func(row):
    if (row['heating'] == 1) | (row['heating'] == 11) | (row['total_solid_fuel'] == 0):
        return 0
    else:
        return get_distinct_values(130, 2)
    
def solid_water_heating_func(row):
    if (row['house_type'] == 1) | (row['house_type'] == 3) | (row['total_solid_fuel'] == 0):
        return 0
    else:
        return get_distinct_values(7, 1)
        
def solid_cooking_func(row):
    if (row['house_type'] == 1) | (row['house_type'] == 3) | (row['total_solid_fuel'] == 0):
        return 0
    else:
        return get_distinct_values(1)

In [47]:
new_category_4['solid_heating'] = new_category_4.apply(solid_heating_func, axis=1)
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'solid_heating'] = 244.4
ALL_VALUES.clear()

new_category_4.loc[(new_category_4.total_air_conditioning >= 0), 'solid_conditioning'] = 0

new_category_4['solid_water_heating'] = new_category_4.apply(solid_water_heating_func, axis=1)
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'solid_water_heating'] = 14.6
ALL_VALUES.clear()

new_category_4['solid_cooking'] = new_category_4.apply(solid_cooking_func, axis=1)
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'solid_cooking'] = 1
ALL_VALUES.clear()

new_category_4.loc[new_category_4.total_PC >= 0, 'solid_lighting'] = 0

new_category_4.loc[new_category_4.total_PC >= 0, 'solid_other'] = 0

In [48]:
def oil_heating_func(row):
    if (row['heating'] == 1) | (row['heating'] == 11) | (row['house_type'] == 1) | (row['house_type'] == 3) | \
       (row['total_liquefied_gas'] == 0):
        return 0
    else:
        return get_distinct_values(4, 1)

In [49]:
new_category_4['oil_heating'] = new_category_4.apply(oil_heating_func, axis=1)
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'oil_heating'] = 7.8
ALL_VALUES.clear()

new_category_4.loc[(new_category_4.total_air_conditioning >= 0), 'oil_conditioning'] = 0

new_category_4.loc[:, 'oil_water_heating'] = new_category_4.total_liquefied_gas.apply(lambda x: 0 if x == 0
                                                                                      else get_distinct_values(5, 1)) 
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'oil_water_heating'] = 8.7
ALL_VALUES.clear()

new_category_4.loc[:, 'oil_cooking'] = new_category_4.total_liquefied_gas.apply(lambda x: 0 if x == 0
                                                                                else get_distinct_values(30, 1)) 
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'oil_cooking'] = 61.5
ALL_VALUES.clear()

new_category_4.loc[new_category_4.total_PC >= 0, 'oil_lighting'] = 0

new_category_4.loc[new_category_4.total_PC >= 0, 'oil_other'] = 0

In [50]:
def bio_heating_func(row):
    if (row['heating'] == 1) | (row['heating'] == 11) | (row['house_type'] == 1) | (row['house_type'] == 3) | \
       (row['total_biofuels'] == 0) | (row['settlement_type'] == 1):
        return 0
    else:
        return get_distinct_values(800, 10)
    
def bio_water_heating_func(row):
    if (row['house_type'] == 1) | (row['house_type'] == 3) | \
       (row['total_biofuels'] == 0) | (row['settlement_type'] == 1):
        return 0
    else:
        return get_distinct_values(30, 1)

In [51]:
new_category_4['bio_heating'] = new_category_4.apply(bio_heating_func, axis=1)
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'bio_heating'] = 1761.394
ALL_VALUES.clear()

new_category_4.loc[(new_category_4.total_air_conditioning >= 0), 'bio_conditioning'] = 0

new_category_4['bio_water_heating'] = new_category_4.apply(bio_water_heating_func, axis=1)
new_category_4.loc[new_category_4.total_number_of_households == 14934921, 'bio_water_heating'] = 52.606
ALL_VALUES.clear()

new_category_4.loc[new_category_4.total_PC >= 0, 'bio_cooking'] = 0

new_category_4.loc[new_category_4.total_PC >= 0, 'bio_lighting'] = 0

new_category_4.loc[new_category_4.total_PC >= 0, 'bio_other'] = 0

In [52]:
new_category_5 = new_category_4.drop(labels=['household_type', 'house_area', 
                                             'total_refrigerator', 'total_freezer', 'total_washing_machine', 
                                             'total_vacuum_cleaner', 'total_colour_TV', 'total_microwave', 
                                             'total_food_processor', 'total_air_conditioning', 'total_dishwasher', 
                                             'total_PC', 'total_laptop', 'total_iron', 'total_electric_stove', 
                                             'total_electric_power', 'total_thermal_energy', 'total_natural_gas',
                                             'total_solid_fuel', 'total_liquefied_gas', 'total_biofuels', 
                                             'total_gas_station'], axis=1)

In [53]:
new_category_5 = new_category_5.round({'el_heating':3, 'el_conditioning':3, 'el_water_heating':3, 'el_cooking':3,
                                       'el_lighting':3, 'el_other':3, 'th_heating':3, 'th_water_heating':3,
                                       'gas_heating':3, 'gas_water_heating':3, 'gas_cooking':3, 'solid_heating':3,
                                       'solid_water_heating':3, 'solid_cooking':3, 'oil_heating':3, 'oil_water_heating':3,
                                       'oil_cooking':3, 'bio_heating':3, 'bio_water_heating':3})

new_category_5[['th_conditioning']] = new_category_5[['th_conditioning']].astype(int)
new_category_5[['th_cooking']] = new_category_5[['th_cooking']].astype(int)
new_category_5[['th_lighting']] = new_category_5[['th_lighting']].astype(int)
new_category_5[['th_other']] = new_category_5[['th_other']].astype(int)
new_category_5[['gas_conditioning']] = new_category_5[['gas_conditioning']].astype(int)
new_category_5[['gas_lighting']] = new_category_5[['gas_lighting']].astype(int)
new_category_5[['gas_other']] = new_category_5[['gas_other']].astype(int)
new_category_5[['solid_conditioning']] = new_category_5[['solid_conditioning']].astype(int)
new_category_5[['solid_lighting']] = new_category_5[['solid_lighting']].astype(int)
new_category_5[['solid_other']] = new_category_5[['solid_other']].astype(int)
new_category_5[['oil_conditioning']] = new_category_5[['oil_conditioning']].astype(int)
new_category_5[['oil_lighting']] = new_category_5[['oil_lighting']].astype(int)
new_category_5[['oil_other']] = new_category_5[['oil_other']].astype(int)
new_category_5[['bio_conditioning']] = new_category_5[['bio_conditioning']].astype(int)
new_category_5[['bio_cooking']] = new_category_5[['bio_cooking']].astype(int)
new_category_5[['bio_lighting']] = new_category_5[['bio_lighting']].astype(int)
new_category_5[['bio_other']] = new_category_5[['bio_other']].astype(int)

In [54]:
import copy

PRECISION = 0.3
SAME_PRECISION = 1e-9
MAX_ITER = 10000

def do_column_estimation(input_matrix: np.ndarray, column_sum: np.ndarray):
    actual_col_sum = np.sum(input_matrix, axis=0)
    col_coef = column_sum / actual_col_sum
    return np.multiply(input_matrix, col_coef)

def do_row_estimation(input_matrix: np.ndarray, row_sum: np.ndarray):
    actual_row_sum = np.sum(input_matrix, axis=1)
    row_coef = row_sum / actual_row_sum
    row_coef = row_coef.reshape((len(row_coef), 1))
    return np.multiply(input_matrix, row_coef)

def iterative_proportional_method(input_matrix: np.ndarray, row_sum: np.ndarray, column_sum: np.ndarray):
    converged = False
    inverse_step = False
    prev_col_diff = np.zeros_like(column_sum)
    prev_row_diff = np.zeros_like(row_sum)
    iter_num = 0
    while not converged:
        if iter_num >= MAX_ITER:
            converged = True
        
        actual_row_sum = np.sum(input_matrix, axis=1)
        actual_col_sum = np.sum(input_matrix, axis=0)

        abs_row_diff = np.abs(row_sum - actual_row_sum)
        abs_col_diff = np.abs(column_sum - actual_col_sum)
        
        if np.all(np.isclose(abs_row_diff, prev_row_diff, atol=SAME_PRECISION)) and np.all(
                np.isclose(abs_col_diff, prev_col_diff, atol=SAME_PRECISION)):
            inverse_step = not inverse_step
            
        prev_row_diff = copy.deepcopy(abs_row_diff)
        prev_col_diff = copy.deepcopy(abs_col_diff)
        
        if np.all(np.isclose(actual_row_sum, row_sum, atol=PRECISION)) and np.all(
                np.isclose(actual_col_sum, column_sum, atol=PRECISION)):
            converged = True
        else:
            iter_num += 1
            if inverse_step:
                input_matrix = do_row_estimation(input_matrix, row_sum)
                input_matrix = do_column_estimation(input_matrix, column_sum)
            else:
                input_matrix = do_column_estimation(input_matrix, column_sum)
                input_matrix = do_row_estimation(input_matrix, row_sum)
    return input_matrix

In [55]:
row_sum = new_category_5['energy_consumption_ttoe'].values[:-1]
column_sum = new_category_5.iloc[-1,6:].values
input_data = new_category_5.iloc[:-1, 6:].values


idx = np.argwhere(np.all(input_data[..., :] == 0, axis=0))
new_column_sum = column_sum[column_sum != 0]
new_input = np.delete(input_data, idx, axis=1)

resulting_table = iterative_proportional_method(new_input, row_sum, new_column_sum)

In [56]:
result_data = np.zeros_like(input_data)
real_id = 0
for column_id in range(input_data.shape[1]):
    if column_id in idx:
        continue
    
    result_data[:, column_id] = resulting_table[:, real_id]
    real_id += 1

In [57]:
new_category_6 = new_category_5.copy(deep=True)
new_category_6.iloc[:-1, 6:] = result_data

In [58]:
new_category_6 = new_category_6.round({'el_heating':3, 'el_conditioning':3, 'el_water_heating':3, 'el_cooking':3,
                                       'el_lighting':3, 'el_other':3, 'th_heating':3, 'th_water_heating':3,
                                       'gas_heating':3, 'gas_water_heating':3, 'gas_cooking':3, 'solid_heating':3,
                                       'solid_water_heating':3, 'solid_cooking':3, 'oil_heating':3, 'oil_water_heating':3,
                                       'oil_cooking':3, 'bio_heating':3, 'bio_water_heating':3})

In [59]:
# new_category_6

In [60]:
el_heating_sum = sum(new_category_6['el_heating'].values[:-1])
th_heating_sum = sum(new_category_6['th_heating'].values[:-1])
gas_heating_sum = sum(new_category_6['gas_heating'].values[:-1])
solid_heating_sum = sum(new_category_6['solid_heating'].values[:-1])
oil_heating_sum = sum(new_category_6['oil_heating'].values[:-1])
bio_heating_sum = sum(new_category_6['bio_heating'].values[:-1])

el_conditioning_sum = sum(new_category_6['el_conditioning'].values[:-1])

el_water_heating_sum = sum(new_category_6['el_water_heating'].values[:-1])
th_water_heating_sum = sum(new_category_6['th_water_heating'].values[:-1])
gas_water_heating_sum = sum(new_category_6['gas_water_heating'].values[:-1])
solid_water_heating_sum = sum(new_category_6['solid_water_heating'].values[:-1])
oil_water_heating_sum = sum(new_category_6['oil_water_heating'].values[:-1])
bio_water_heating_sum = sum(new_category_6['bio_water_heating'].values[:-1])

el_cooking_sum = sum(new_category_6['el_cooking'].values[:-1])
gas_cooking_sum = sum(new_category_6['gas_cooking'].values[:-1])
solid_cooking_sum = sum(new_category_6['solid_cooking'].values[:-1])
oil_cooking_sum = sum(new_category_6['oil_cooking'].values[:-1])

el_lighting_sum = sum(new_category_6['el_lighting'].values[:-1])

el_other_sum = sum(new_category_6['el_other'].values[:-1])

In [61]:
final_table_1 = pd.DataFrame({'heating': [0, 0, 0, 0, 0, 0], 'conditioning': [0, 0, 0, 0, 0, 0], 
                              'water_heating': [0, 0, 0, 0, 0, 0], 'cooking': [0, 0, 0, 0, 0, 0],
                              'lighting': [0, 0, 0, 0, 0, 0], 'other': [0, 0, 0, 0, 0, 0]})

In [62]:
final_table_1.loc[0, 'heating'] = el_heating_sum
final_table_1.loc[1, 'heating'] = th_heating_sum
final_table_1.loc[2, 'heating'] = gas_heating_sum
final_table_1.loc[3, 'heating'] = solid_heating_sum
final_table_1.loc[4, 'heating'] = oil_heating_sum
final_table_1.loc[5, 'heating'] = bio_heating_sum

final_table_1.loc[0, 'conditioning'] = el_conditioning_sum

final_table_1.loc[0, 'water_heating'] = el_water_heating_sum
final_table_1.loc[1, 'water_heating'] = th_water_heating_sum
final_table_1.loc[2, 'water_heating'] = gas_water_heating_sum
final_table_1.loc[3, 'water_heating'] = solid_water_heating_sum
final_table_1.loc[4, 'water_heating'] = oil_water_heating_sum
final_table_1.loc[5, 'water_heating'] = bio_water_heating_sum

final_table_1.loc[0, 'cooking'] = el_cooking_sum
final_table_1.loc[2, 'cooking'] = gas_cooking_sum
final_table_1.loc[3, 'cooking'] = solid_cooking_sum
final_table_1.loc[4, 'cooking'] = oil_cooking_sum

final_table_1.loc[0, 'lighting'] = el_lighting_sum

final_table_1.loc[0, 'other'] = el_other_sum

In [63]:
final_table_1['total'] = final_table_1.heating + final_table_1.conditioning + final_table_1.water_heating + \
                         final_table_1.cooking + final_table_1.lighting + final_table_1.other

In [64]:
final_table_1['category'] = (['electricity', 'thermal_energy', 'gas', 'solid_fuel', 'oil', 'bio_fuel'])

final_table_1 = final_table_1.reset_index().set_index('category')

final_table_1 = final_table_1.drop(labels='index', axis=1)

In [65]:
final_table_1.loc['sum'] = [0, 0, 0, 0, 0, 0, 0]

final_table_1.loc['sum', 'heating'] = sum(final_table_1['heating'].values)
final_table_1.loc['sum', 'conditioning'] = sum(final_table_1['conditioning'].values)
final_table_1.loc['sum', 'water_heating'] = sum(final_table_1['water_heating'].values)
final_table_1.loc['sum', 'cooking'] = sum(final_table_1['cooking'].values)
final_table_1.loc['sum', 'lighting'] = sum(final_table_1['lighting'].values)
final_table_1.loc['sum', 'other'] = sum(final_table_1['other'].values)
final_table_1.loc['sum', 'total'] = sum(final_table_1['total'].values)

In [66]:
# final_table_1

In [67]:
final_table_2 = final_table_1.round({'heating':2, 'conditioning':2, 'water_heating':2, 'cooking':2, 'lighting':2,
                                     'other': 2, 'total': 2})

In [68]:
heating_1 = final_table_2.loc['electricity','heating'] * 100 / final_table_2.loc['electricity','total']
heating_2 = final_table_2.loc['thermal_energy','heating'] * 100 / final_table_2.loc['thermal_energy','total']
heating_3 = final_table_2.loc['gas','heating'] * 100 / final_table_2.loc['gas','total']
heating_4 = final_table_2.loc['solid_fuel','heating'] * 100 / final_table_2.loc['solid_fuel','total']
heating_5 = final_table_2.loc['oil','heating'] * 100 / final_table_2.loc['oil','total']
heating_6 = final_table_2.loc['bio_fuel','heating'] * 100 / final_table_2.loc['bio_fuel','total']
heating_7 = final_table_2.loc['sum','heating'] * 100 / final_table_2.loc['sum','total']

conditioning_1 = final_table_2.loc['electricity','conditioning'] * 100 / final_table_2.loc['electricity','total']
conditioning_7 = final_table_2.loc['sum','conditioning'] * 100 / final_table_2.loc['sum','total']

w_heating_1 = final_table_2.loc['electricity','water_heating'] * 100 / final_table_2.loc['electricity','total']
w_heating_2 = final_table_2.loc['thermal_energy','water_heating'] * 100 / final_table_2.loc['thermal_energy','total']
w_heating_3 = final_table_2.loc['gas','water_heating'] * 100 / final_table_2.loc['gas','total']
w_heating_4 = final_table_2.loc['solid_fuel','water_heating'] * 100 / final_table_2.loc['solid_fuel','total']
w_heating_5 = final_table_2.loc['oil','water_heating'] * 100 / final_table_2.loc['oil','total']
w_heating_6 = final_table_2.loc['bio_fuel','water_heating'] * 100 / final_table_2.loc['bio_fuel','total']
w_heating_7 = final_table_2.loc['sum','water_heating'] * 100 / final_table_2.loc['sum','total']

cooking_1 = final_table_2.loc['electricity','cooking'] * 100 / final_table_2.loc['electricity','total']
cooking_3 = final_table_2.loc['gas','cooking'] * 100 / final_table_2.loc['gas','total']
cooking_4 = final_table_2.loc['solid_fuel','cooking'] * 100 / final_table_2.loc['solid_fuel','total']
cooking_5 = final_table_2.loc['oil','cooking'] * 100 / final_table_2.loc['oil','total']
cooking_7 = final_table_2.loc['sum','cooking'] * 100 / final_table_2.loc['sum','total']

lighting_1 = final_table_2.loc['electricity','lighting'] * 100 / final_table_2.loc['electricity','total']
lighting_7 = final_table_2.loc['sum','lighting'] * 100 / final_table_2.loc['sum','total']

other_1 = final_table_2.loc['electricity','other'] * 100 / final_table_2.loc['electricity','total']
other_7 = final_table_2.loc['sum','other'] * 100 / final_table_2.loc['sum','total']

In [69]:
final_table_2.loc['electricity','heating'] = heating_1
final_table_2.loc['thermal_energy','heating'] = heating_2
final_table_2.loc['gas','heating'] = heating_3
final_table_2.loc['solid_fuel','heating'] = heating_4
final_table_2.loc['oil','heating'] = heating_5
final_table_2.loc['bio_fuel','heating'] = heating_6
final_table_2.loc['sum','heating'] = heating_7

final_table_2.loc['electricity','conditioning'] = conditioning_1
final_table_2.loc['sum','conditioning'] = conditioning_7

final_table_2.loc['electricity','water_heating'] = w_heating_1
final_table_2.loc['thermal_energy','water_heating'] = w_heating_2
final_table_2.loc['gas','water_heating'] = w_heating_3
final_table_2.loc['solid_fuel','water_heating'] = w_heating_4
final_table_2.loc['oil','water_heating'] = w_heating_5
final_table_2.loc['bio_fuel','water_heating'] = w_heating_6
final_table_2.loc['sum','water_heating'] = w_heating_7

final_table_2.loc['electricity','cooking'] = cooking_1
final_table_2.loc['gas','cooking'] = cooking_3
final_table_2.loc['solid_fuel','cooking'] = cooking_4
final_table_2.loc['oil','cooking'] = cooking_5
final_table_2.loc['sum','cooking'] = cooking_7

final_table_2.loc['electricity','lighting'] = lighting_1
final_table_2.loc['sum','lighting'] = lighting_7

final_table_2.loc['electricity','other'] = other_1
final_table_2.loc['sum','other'] = other_7

In [70]:
final_table_2['total'] = [100, 100, 100, 100, 100, 100, 100]

In [71]:
final_table_2 = final_table_2.round({'heating':2, 'conditioning':2, 'water_heating':2, 'cooking':2, 'lighting':2,
                                     'other': 2, 'total': 2})

In [72]:
# final_table_2

In [73]:
# final_table_1.to_excel("ttoe.xlsx")
# final_table_2.to_excel("percent.xlsx")

In [74]:
new_category_7 = new_category_6.copy(deep = True)

In [75]:
new_category_7.loc[new_category_7.settlement_type == 2,'el_heating'] = 0
new_category_7.loc[new_category_7.settlement_type == 2,'el_conditioning'] = 0
new_category_7.loc[new_category_7.settlement_type == 2,'el_water_heating'] = 0
new_category_7.loc[new_category_7.settlement_type == 2,'el_cooking'] = 0
new_category_7.loc[new_category_7.settlement_type == 2,'el_lighting'] = 0
new_category_7.loc[new_category_7.settlement_type == 2,'el_other'] = 0
new_category_7.loc[new_category_7.settlement_type == 2,'th_heating'] = 0
new_category_7.loc[new_category_7.settlement_type == 2,'th_water_heating'] = 0
new_category_7.loc[new_category_7.settlement_type == 2,'gas_heating'] = 0
new_category_7.loc[new_category_7.settlement_type == 2,'gas_water_heating'] = 0
new_category_7.loc[new_category_7.settlement_type == 2,'gas_cooking'] = 0
new_category_7.loc[new_category_7.settlement_type == 2,'solid_heating'] = 0
new_category_7.loc[new_category_7.settlement_type == 2,'solid_water_heating'] = 0
new_category_7.loc[new_category_7.settlement_type == 2,'solid_cooking'] = 0
new_category_7.loc[new_category_7.settlement_type == 2,'oil_heating'] = 0
new_category_7.loc[new_category_7.settlement_type == 2,'oil_water_heating'] = 0
new_category_7.loc[new_category_7.settlement_type == 2,'oil_cooking'] = 0
new_category_7.loc[new_category_7.settlement_type == 2,'bio_heating'] = 0
new_category_7.loc[new_category_7.settlement_type == 2,'bio_water_heating'] = 0

In [76]:
el_heating_sum = sum(new_category_7['el_heating'].values[:-1])
th_heating_sum = sum(new_category_7['th_heating'].values[:-1])
gas_heating_sum = sum(new_category_7['gas_heating'].values[:-1])
solid_heating_sum = sum(new_category_7['solid_heating'].values[:-1])
oil_heating_sum = sum(new_category_7['oil_heating'].values[:-1])
bio_heating_sum = sum(new_category_7['bio_heating'].values[:-1])

el_conditioning_sum = sum(new_category_7['el_conditioning'].values[:-1])

el_water_heating_sum = sum(new_category_7['el_water_heating'].values[:-1])
th_water_heating_sum = sum(new_category_7['th_water_heating'].values[:-1])
gas_water_heating_sum = sum(new_category_7['gas_water_heating'].values[:-1])
solid_water_heating_sum = sum(new_category_7['solid_water_heating'].values[:-1])
oil_water_heating_sum = sum(new_category_7['oil_water_heating'].values[:-1])
bio_water_heating_sum = sum(new_category_7['bio_water_heating'].values[:-1])

el_cooking_sum = sum(new_category_7['el_cooking'].values[:-1])
gas_cooking_sum = sum(new_category_7['gas_cooking'].values[:-1])
solid_cooking_sum = sum(new_category_7['solid_cooking'].values[:-1])
oil_cooking_sum = sum(new_category_7['oil_cooking'].values[:-1])

el_lighting_sum = sum(new_category_7['el_lighting'].values[:-1])

el_other_sum = sum(new_category_7['el_other'].values[:-1])

In [77]:
final_table_3 = pd.DataFrame({'heating': [0, 0, 0, 0, 0, 0], 'conditioning': [0, 0, 0, 0, 0, 0], 
                              'water_heating': [0, 0, 0, 0, 0, 0], 'cooking': [0, 0, 0, 0, 0, 0],
                              'lighting': [0, 0, 0, 0, 0, 0], 'other': [0, 0, 0, 0, 0, 0]})

In [78]:
final_table_3.loc[0, 'heating'] = el_heating_sum
final_table_3.loc[1, 'heating'] = th_heating_sum
final_table_3.loc[2, 'heating'] = gas_heating_sum
final_table_3.loc[3, 'heating'] = solid_heating_sum
final_table_3.loc[4, 'heating'] = oil_heating_sum
final_table_3.loc[5, 'heating'] = bio_heating_sum

final_table_3.loc[0, 'conditioning'] = el_conditioning_sum

final_table_3.loc[0, 'water_heating'] = el_water_heating_sum
final_table_3.loc[1, 'water_heating'] = th_water_heating_sum
final_table_3.loc[2, 'water_heating'] = gas_water_heating_sum
final_table_3.loc[3, 'water_heating'] = solid_water_heating_sum
final_table_3.loc[4, 'water_heating'] = oil_water_heating_sum
final_table_3.loc[5, 'water_heating'] = bio_water_heating_sum

final_table_3.loc[0, 'cooking'] = el_cooking_sum
final_table_3.loc[2, 'cooking'] = gas_cooking_sum
final_table_3.loc[3, 'cooking'] = solid_cooking_sum
final_table_3.loc[4, 'cooking'] = oil_cooking_sum

final_table_3.loc[0, 'lighting'] = el_lighting_sum

final_table_3.loc[0, 'other'] = el_other_sum

In [79]:
final_table_3['total'] = final_table_3.heating + final_table_3.conditioning + final_table_3.water_heating + \
                         final_table_3.cooking + final_table_3.lighting + final_table_3.other

In [80]:
final_table_3['category'] = (['electricity', 'thermal_energy', 'gas', 'solid_fuel', 'oil', 'bio_fuel'])

final_table_3 = final_table_3.reset_index().set_index('category')

final_table_3 = final_table_3.drop(labels='index', axis=1)

In [81]:
final_table_3.loc['sum'] = [0, 0, 0, 0, 0, 0, 0]

final_table_3.loc['sum', 'heating'] = sum(final_table_3['heating'].values)
final_table_3.loc['sum', 'conditioning'] = sum(final_table_3['conditioning'].values)
final_table_3.loc['sum', 'water_heating'] = sum(final_table_3['water_heating'].values)
final_table_3.loc['sum', 'cooking'] = sum(final_table_3['cooking'].values)
final_table_3.loc['sum', 'lighting'] = sum(final_table_3['lighting'].values)
final_table_3.loc['sum', 'other'] = sum(final_table_3['other'].values)
final_table_3.loc['sum', 'total'] = sum(final_table_3['total'].values)

In [82]:
final_city = final_table_3.copy(deep = True)

In [83]:
# final_city

In [84]:
final_city_1 = final_city.round({'heating':2, 'conditioning':2, 'water_heating':2, 'cooking':2, 'lighting':2,
                                 'other': 2, 'total': 2})

In [85]:
heating_1 = final_city_1.loc['electricity','heating'] * 100 / final_city_1.loc['electricity','total']
heating_2 = final_city_1.loc['thermal_energy','heating'] * 100 / final_city_1.loc['thermal_energy','total']
heating_3 = final_city_1.loc['gas','heating'] * 100 / final_city_1.loc['gas','total']
heating_4 = final_city_1.loc['solid_fuel','heating'] * 100 / final_city_1.loc['solid_fuel','total']
heating_5 = final_city_1.loc['oil','heating'] * 100 / final_city_1.loc['oil','total']
heating_7 = final_city_1.loc['sum','heating'] * 100 / final_city_1.loc['sum','total']

conditioning_1 = final_city_1.loc['electricity','conditioning'] * 100 / final_city_1.loc['electricity','total']
conditioning_7 = final_city_1.loc['sum','conditioning'] * 100 / final_city_1.loc['sum','total']

w_heating_1 = final_city_1.loc['electricity','water_heating'] * 100 / final_city_1.loc['electricity','total']
w_heating_2 = final_city_1.loc['thermal_energy','water_heating'] * 100 / final_city_1.loc['thermal_energy','total']
w_heating_3 = final_city_1.loc['gas','water_heating'] * 100 / final_city_1.loc['gas','total']
w_heating_4 = final_city_1.loc['solid_fuel','water_heating'] * 100 / final_city_1.loc['solid_fuel','total']
w_heating_5 = final_city_1.loc['oil','water_heating'] * 100 / final_city_1.loc['oil','total']
w_heating_7 = final_city_1.loc['sum','water_heating'] * 100 / final_city_1.loc['sum','total']

cooking_1 = final_city_1.loc['electricity','cooking'] * 100 / final_city_1.loc['electricity','total']
cooking_3 = final_city_1.loc['gas','cooking'] * 100 / final_city_1.loc['gas','total']
cooking_4 = final_city_1.loc['solid_fuel','cooking'] * 100 / final_city_1.loc['solid_fuel','total']
cooking_5 = final_city_1.loc['oil','cooking'] * 100 / final_city_1.loc['oil','total']
cooking_7 = final_city_1.loc['sum','cooking'] * 100 / final_city_1.loc['sum','total']

lighting_1 = final_city_1.loc['electricity','lighting'] * 100 / final_city_1.loc['electricity','total']
lighting_7 = final_city_1.loc['sum','lighting'] * 100 / final_city_1.loc['sum','total']

other_1 = final_city_1.loc['electricity','other'] * 100 / final_city_1.loc['electricity','total']
other_7 = final_city_1.loc['sum','other'] * 100 / final_city_1.loc['sum','total']

In [86]:
final_city_1.loc['electricity','heating'] = heating_1
final_city_1.loc['thermal_energy','heating'] = heating_2
final_city_1.loc['gas','heating'] = heating_3
final_city_1.loc['solid_fuel','heating'] = heating_4
final_city_1.loc['oil','heating'] = heating_5
final_city_1.loc['sum','heating'] = heating_7

final_city_1.loc['electricity','conditioning'] = conditioning_1
final_city_1.loc['sum','conditioning'] = conditioning_7

final_city_1.loc['electricity','water_heating'] = w_heating_1
final_city_1.loc['thermal_energy','water_heating'] = w_heating_2
final_city_1.loc['gas','water_heating'] = w_heating_3
final_city_1.loc['solid_fuel','water_heating'] = w_heating_4
final_city_1.loc['oil','water_heating'] = w_heating_5
final_city_1.loc['sum','water_heating'] = w_heating_7

final_city_1.loc['electricity','cooking'] = cooking_1
final_city_1.loc['gas','cooking'] = cooking_3
final_city_1.loc['solid_fuel','cooking'] = cooking_4
final_city_1.loc['oil','cooking'] = cooking_5
final_city_1.loc['sum','cooking'] = cooking_7

final_city_1.loc['electricity','lighting'] = lighting_1
final_city_1.loc['sum','lighting'] = lighting_7

final_city_1.loc['electricity','other'] = other_1
final_city_1.loc['sum','other'] = other_7

In [87]:
final_city_1['total'] = [100, 100, 100, 100, 100, 0, 100]

In [88]:
final_city_1 = final_city_1.round({'heating':2, 'conditioning':2, 'water_heating':2, 'cooking':2, 'lighting':2,
                                    'other': 2, 'total': 2})

In [89]:
# final_city_1

In [90]:
new_category_8 = new_category_6.copy(deep = True)

In [91]:
new_category_8.loc[new_category_8.settlement_type == 1,'el_heating'] = 0
new_category_8.loc[new_category_8.settlement_type == 1,'el_conditioning'] = 0
new_category_8.loc[new_category_8.settlement_type == 1,'el_water_heating'] = 0
new_category_8.loc[new_category_8.settlement_type == 1,'el_cooking'] = 0
new_category_8.loc[new_category_8.settlement_type == 1,'el_lighting'] = 0
new_category_8.loc[new_category_8.settlement_type == 1,'el_other'] = 0
new_category_8.loc[new_category_8.settlement_type == 1,'th_heating'] = 0
new_category_8.loc[new_category_8.settlement_type == 1,'th_water_heating'] = 0
new_category_8.loc[new_category_8.settlement_type == 1,'gas_heating'] = 0
new_category_8.loc[new_category_8.settlement_type == 1,'gas_water_heating'] = 0
new_category_8.loc[new_category_8.settlement_type == 1,'gas_cooking'] = 0
new_category_8.loc[new_category_8.settlement_type == 1,'solid_heating'] = 0
new_category_8.loc[new_category_8.settlement_type == 1,'solid_water_heating'] = 0
new_category_8.loc[new_category_8.settlement_type == 1,'solid_cooking'] = 0
new_category_8.loc[new_category_8.settlement_type == 1,'oil_heating'] = 0
new_category_8.loc[new_category_8.settlement_type == 1,'oil_water_heating'] = 0
new_category_8.loc[new_category_8.settlement_type == 1,'oil_cooking'] = 0
new_category_8.loc[new_category_8.settlement_type == 1,'bio_heating'] = 0
new_category_8.loc[new_category_8.settlement_type == 1,'bio_water_heating'] = 0

In [92]:
el_heating_sum = sum(new_category_8['el_heating'].values[:-1])
th_heating_sum = sum(new_category_8['th_heating'].values[:-1])
gas_heating_sum = sum(new_category_8['gas_heating'].values[:-1])
solid_heating_sum = sum(new_category_8['solid_heating'].values[:-1])
oil_heating_sum = sum(new_category_8['oil_heating'].values[:-1])
bio_heating_sum = sum(new_category_8['bio_heating'].values[:-1])

el_conditioning_sum = sum(new_category_8['el_conditioning'].values[:-1])

el_water_heating_sum = sum(new_category_8['el_water_heating'].values[:-1])
th_water_heating_sum = sum(new_category_8['th_water_heating'].values[:-1])
gas_water_heating_sum = sum(new_category_8['gas_water_heating'].values[:-1])
solid_water_heating_sum = sum(new_category_8['solid_water_heating'].values[:-1])
oil_water_heating_sum = sum(new_category_8['oil_water_heating'].values[:-1])
bio_water_heating_sum = sum(new_category_8['bio_water_heating'].values[:-1])

el_cooking_sum = sum(new_category_8['el_cooking'].values[:-1])
gas_cooking_sum = sum(new_category_8['gas_cooking'].values[:-1])
solid_cooking_sum = sum(new_category_8['solid_cooking'].values[:-1])
oil_cooking_sum = sum(new_category_8['oil_cooking'].values[:-1])

el_lighting_sum = sum(new_category_8['el_lighting'].values[:-1])

el_other_sum = sum(new_category_8['el_other'].values[:-1])

In [93]:
final_table_4 = pd.DataFrame({'heating': [0, 0, 0, 0, 0, 0], 'conditioning': [0, 0, 0, 0, 0, 0], 
                              'water_heating': [0, 0, 0, 0, 0, 0], 'cooking': [0, 0, 0, 0, 0, 0],
                              'lighting': [0, 0, 0, 0, 0, 0], 'other': [0, 0, 0, 0, 0, 0]})

In [94]:
final_table_4.loc[0, 'heating'] = el_heating_sum
final_table_4.loc[1, 'heating'] = th_heating_sum
final_table_4.loc[2, 'heating'] = gas_heating_sum
final_table_4.loc[3, 'heating'] = solid_heating_sum
final_table_4.loc[4, 'heating'] = oil_heating_sum
final_table_4.loc[5, 'heating'] = bio_heating_sum

final_table_4.loc[0, 'conditioning'] = el_conditioning_sum

final_table_4.loc[0, 'water_heating'] = el_water_heating_sum
final_table_4.loc[1, 'water_heating'] = th_water_heating_sum
final_table_4.loc[2, 'water_heating'] = gas_water_heating_sum
final_table_4.loc[3, 'water_heating'] = solid_water_heating_sum
final_table_4.loc[4, 'water_heating'] = oil_water_heating_sum
final_table_4.loc[5, 'water_heating'] = bio_water_heating_sum

final_table_4.loc[0, 'cooking'] = el_cooking_sum
final_table_4.loc[2, 'cooking'] = gas_cooking_sum
final_table_4.loc[3, 'cooking'] = solid_cooking_sum
final_table_4.loc[4, 'cooking'] = oil_cooking_sum

final_table_4.loc[0, 'lighting'] = el_lighting_sum

final_table_4.loc[0, 'other'] = el_other_sum

In [95]:
final_table_4['total'] = final_table_4.heating + final_table_4.conditioning + final_table_4.water_heating + \
                         final_table_4.cooking + final_table_4.lighting + final_table_4.other

In [96]:
final_table_4['category'] = (['electricity', 'thermal_energy', 'gas', 'solid_fuel', 'oil', 'bio_fuel'])

final_table_4 = final_table_4.reset_index().set_index('category')

final_table_4 = final_table_4.drop(labels='index', axis=1)

In [97]:
final_table_4.loc['sum'] = [0, 0, 0, 0, 0, 0, 0]

final_table_4.loc['sum', 'heating'] = sum(final_table_4['heating'].values)
final_table_4.loc['sum', 'conditioning'] = sum(final_table_4['conditioning'].values)
final_table_4.loc['sum', 'water_heating'] = sum(final_table_4['water_heating'].values)
final_table_4.loc['sum', 'cooking'] = sum(final_table_4['cooking'].values)
final_table_4.loc['sum', 'lighting'] = sum(final_table_4['lighting'].values)
final_table_4.loc['sum', 'other'] = sum(final_table_4['other'].values)
final_table_4.loc['sum', 'total'] = sum(final_table_4['total'].values)

In [98]:
final_village = final_table_4.copy(deep = True)

In [99]:
# final_village

In [100]:
final_village_1 = final_village.round({'heating':2, 'conditioning':2, 'water_heating':2, 'cooking':2, 'lighting':2,
                                       'other': 2, 'total': 2})

In [101]:
heating_1 = final_village_1.loc['electricity','heating'] * 100 / final_village_1.loc['electricity','total']
heating_2 = final_village_1.loc['thermal_energy','heating'] * 100 / final_village_1.loc['thermal_energy','total']
heating_3 = final_village_1.loc['gas','heating'] * 100 / final_village_1.loc['gas','total']
heating_4 = final_village_1.loc['solid_fuel','heating'] * 100 / final_village_1.loc['solid_fuel','total']
heating_5 = final_village_1.loc['oil','heating'] * 100 / final_village_1.loc['oil','total']
heating_6 = final_village_1.loc['bio_fuel','heating'] * 100 / final_village_1.loc['bio_fuel','total']
heating_7 = final_village_1.loc['sum','heating'] * 100 / final_village_1.loc['sum','total']

conditioning_1 = final_village_1.loc['electricity','conditioning'] * 100 / final_village_1.loc['electricity','total']
conditioning_7 = final_village_1.loc['sum','conditioning'] * 100 / final_village_1.loc['sum','total']

w_heating_1 = final_village_1.loc['electricity','water_heating'] * 100 / final_village_1.loc['electricity','total']
w_heating_2 = final_village_1.loc['thermal_energy','water_heating'] * 100 / final_village_1.loc['thermal_energy','total']
w_heating_3 = final_village_1.loc['gas','water_heating'] * 100 / final_village_1.loc['gas','total']
w_heating_4 = final_village_1.loc['solid_fuel','water_heating'] * 100 / final_village_1.loc['solid_fuel','total']
w_heating_5 = final_village_1.loc['oil','water_heating'] * 100 / final_village_1.loc['oil','total']
w_heating_6 = final_village_1.loc['bio_fuel','water_heating'] * 100 / final_village_1.loc['bio_fuel','total']
w_heating_7 = final_village_1.loc['sum','water_heating'] * 100 / final_village_1.loc['sum','total']

cooking_1 = final_village_1.loc['electricity','cooking'] * 100 / final_village_1.loc['electricity','total']
cooking_3 = final_village_1.loc['gas','cooking'] * 100 / final_village_1.loc['gas','total']
cooking_4 = final_village_1.loc['solid_fuel','cooking'] * 100 / final_village_1.loc['solid_fuel','total']
cooking_5 = final_village_1.loc['oil','cooking'] * 100 / final_village_1.loc['oil','total']
cooking_7 = final_village_1.loc['sum','cooking'] * 100 / final_village_1.loc['sum','total']

lighting_1 = final_village_1.loc['electricity','lighting'] * 100 / final_village_1.loc['electricity','total']
lighting_7 = final_village_1.loc['sum','lighting'] * 100 / final_village_1.loc['sum','total']

other_1 = final_village_1.loc['electricity','other'] * 100 / final_village_1.loc['electricity','total']
other_7 = final_village_1.loc['sum','other'] * 100 / final_village_1.loc['sum','total']

In [102]:
final_village_1.loc['electricity','heating'] = heating_1
final_village_1.loc['thermal_energy','heating'] = heating_2
final_village_1.loc['gas','heating'] = heating_3
final_village_1.loc['solid_fuel','heating'] = heating_4
final_village_1.loc['oil','heating'] = heating_5
final_village_1.loc['bio_fuel','heating'] = heating_6
final_village_1.loc['sum','heating'] = heating_7

final_village_1.loc['electricity','conditioning'] = conditioning_1
final_village_1.loc['sum','conditioning'] = conditioning_7

final_village_1.loc['electricity','water_heating'] = w_heating_1
final_village_1.loc['thermal_energy','water_heating'] = w_heating_2
final_village_1.loc['gas','water_heating'] = w_heating_3
final_village_1.loc['solid_fuel','water_heating'] = w_heating_4
final_village_1.loc['oil','water_heating'] = w_heating_5
final_village_1.loc['bio_fuel','water_heating'] = w_heating_6
final_village_1.loc['sum','water_heating'] = w_heating_7

final_village_1.loc['electricity','cooking'] = cooking_1
final_village_1.loc['gas','cooking'] = cooking_3
final_village_1.loc['solid_fuel','cooking'] = cooking_4
final_village_1.loc['oil','cooking'] = cooking_5
final_village_1.loc['sum','cooking'] = cooking_7

final_village_1.loc['electricity','lighting'] = lighting_1
final_village_1.loc['sum','lighting'] = lighting_7

final_village_1.loc['electricity','other'] = other_1
final_village_1.loc['sum','other'] = other_7

In [103]:
final_village_1['total'] = [100, 100, 100, 100, 100, 100, 100]

In [104]:
final_village_1 = final_village_1.round({'heating':2, 'conditioning':2, 'water_heating':2, 'cooking':2, 'lighting':2,
                                         'other': 2, 'total': 2})

In [105]:
# final_village_1

In [106]:
# final_city_1.to_excel("city_per.xlsx")
# final_village_1.to_excel("village_per.xlsx")

In [107]:
# final_city.to_excel("city.xlsx")
# final_village.to_excel("village.xlsx")

In [108]:
new_category_9 = new_category_6.copy(deep = True)

In [109]:
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'el_heating'] = 0
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'el_conditioning'] = 0
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'el_water_heating'] = 0
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'el_cooking'] = 0
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'el_lighting'] = 0
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'el_other'] = 0
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'th_heating'] = 0
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'th_water_heating'] = 0
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'gas_heating'] = 0
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'gas_water_heating'] = 0
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'gas_cooking'] = 0
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'solid_heating'] = 0
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'solid_water_heating'] = 0
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'solid_cooking'] = 0
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'oil_heating'] = 0
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'oil_water_heating'] = 0
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'oil_cooking'] = 0
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'bio_heating'] = 0
new_category_9.loc[((new_category_9.house_type == 1) | (new_category_9.house_type == 3) | \
                    (new_category_9.house_type == 11) | (new_category_9.house_type == 22) | \
                    ((new_category_9.house_type == 2) & (new_category_9.heating == 1))),'bio_water_heating'] = 0

In [110]:
el_heating_sum = sum(new_category_9['el_heating'].values[:-1])
th_heating_sum = sum(new_category_9['th_heating'].values[:-1])
gas_heating_sum = sum(new_category_9['gas_heating'].values[:-1])
solid_heating_sum = sum(new_category_9['solid_heating'].values[:-1])
oil_heating_sum = sum(new_category_9['oil_heating'].values[:-1])
bio_heating_sum = sum(new_category_9['bio_heating'].values[:-1])

el_conditioning_sum = sum(new_category_9['el_conditioning'].values[:-1])

el_water_heating_sum = sum(new_category_9['el_water_heating'].values[:-1])
th_water_heating_sum = sum(new_category_9['th_water_heating'].values[:-1])
gas_water_heating_sum = sum(new_category_9['gas_water_heating'].values[:-1])
solid_water_heating_sum = sum(new_category_9['solid_water_heating'].values[:-1])
oil_water_heating_sum = sum(new_category_9['oil_water_heating'].values[:-1])
bio_water_heating_sum = sum(new_category_9['bio_water_heating'].values[:-1])

el_cooking_sum = sum(new_category_9['el_cooking'].values[:-1])
gas_cooking_sum = sum(new_category_9['gas_cooking'].values[:-1])
solid_cooking_sum = sum(new_category_9['solid_cooking'].values[:-1])
oil_cooking_sum = sum(new_category_9['oil_cooking'].values[:-1])

el_lighting_sum = sum(new_category_9['el_lighting'].values[:-1])

el_other_sum = sum(new_category_9['el_other'].values[:-1])

In [111]:
final_table_5 = pd.DataFrame({'heating': [0, 0, 0, 0, 0, 0], 'conditioning': [0, 0, 0, 0, 0, 0], 
                              'water_heating': [0, 0, 0, 0, 0, 0], 'cooking': [0, 0, 0, 0, 0, 0],
                              'lighting': [0, 0, 0, 0, 0, 0], 'other': [0, 0, 0, 0, 0, 0]})

In [112]:
final_table_5.loc[0, 'heating'] = el_heating_sum
final_table_5.loc[1, 'heating'] = th_heating_sum
final_table_5.loc[2, 'heating'] = gas_heating_sum
final_table_5.loc[3, 'heating'] = solid_heating_sum
final_table_5.loc[4, 'heating'] = oil_heating_sum
final_table_5.loc[5, 'heating'] = bio_heating_sum

final_table_5.loc[0, 'conditioning'] = el_conditioning_sum

final_table_5.loc[0, 'water_heating'] = el_water_heating_sum
final_table_5.loc[1, 'water_heating'] = th_water_heating_sum
final_table_5.loc[2, 'water_heating'] = gas_water_heating_sum
final_table_5.loc[3, 'water_heating'] = solid_water_heating_sum
final_table_5.loc[4, 'water_heating'] = oil_water_heating_sum
final_table_5.loc[5, 'water_heating'] = bio_water_heating_sum

final_table_5.loc[0, 'cooking'] = el_cooking_sum
final_table_5.loc[2, 'cooking'] = gas_cooking_sum
final_table_5.loc[3, 'cooking'] = solid_cooking_sum
final_table_5.loc[4, 'cooking'] = oil_cooking_sum

final_table_5.loc[0, 'lighting'] = el_lighting_sum

final_table_5.loc[0, 'other'] = el_other_sum

In [113]:
final_table_5['total'] = final_table_5.heating + final_table_5.conditioning + final_table_5.water_heating + \
                         final_table_5.cooking + final_table_5.lighting + final_table_5.other

In [114]:
final_table_5['category'] = (['electricity', 'thermal_energy', 'gas', 'solid_fuel', 'oil', 'bio_fuel'])

final_table_5 = final_table_5.reset_index().set_index('category')

final_table_5 = final_table_5.drop(labels='index', axis=1)

In [115]:
final_table_5.loc['sum'] = [0, 0, 0, 0, 0, 0, 0]

final_table_5.loc['sum', 'heating'] = sum(final_table_5['heating'].values)
final_table_5.loc['sum', 'conditioning'] = sum(final_table_5['conditioning'].values)
final_table_5.loc['sum', 'water_heating'] = sum(final_table_5['water_heating'].values)
final_table_5.loc['sum', 'cooking'] = sum(final_table_5['cooking'].values)
final_table_5.loc['sum', 'lighting'] = sum(final_table_5['lighting'].values)
final_table_5.loc['sum', 'other'] = sum(final_table_5['other'].values)
final_table_5.loc['sum', 'total'] = sum(final_table_5['total'].values)

In [116]:
final_house = final_table_5.copy(deep = True)

In [117]:
# final_house

In [118]:
final_house_1 = final_house.round({'heating':2, 'conditioning':2, 'water_heating':2, 'cooking':2, 'lighting':2,
                                   'other': 2, 'total': 2})

In [119]:
heating_1 = final_house_1.loc['electricity','heating'] * 100 / final_house_1.loc['electricity','total']
heating_3 = final_house_1.loc['gas','heating'] * 100 / final_house_1.loc['gas','total']
heating_4 = final_house_1.loc['solid_fuel','heating'] * 100 / final_house_1.loc['solid_fuel','total']
heating_5 = final_house_1.loc['oil','heating'] * 100 / final_house_1.loc['oil','total']
heating_6 = final_house_1.loc['bio_fuel','heating'] * 100 / final_house_1.loc['bio_fuel','total']
heating_7 = final_house_1.loc['sum','heating'] * 100 / final_house_1.loc['sum','total']

conditioning_1 = final_house_1.loc['electricity','conditioning'] * 100 / final_house_1.loc['electricity','total']
conditioning_7 = final_house_1.loc['sum','conditioning'] * 100 / final_house_1.loc['sum','total']

w_heating_1 = final_house_1.loc['electricity','water_heating'] * 100 / final_house_1.loc['electricity','total']
w_heating_3 = final_house_1.loc['gas','water_heating'] * 100 / final_house_1.loc['gas','total']
w_heating_4 = final_house_1.loc['solid_fuel','water_heating'] * 100 / final_house_1.loc['solid_fuel','total']
w_heating_5 = final_house_1.loc['oil','water_heating'] * 100 / final_house_1.loc['oil','total']
w_heating_6 = final_house_1.loc['bio_fuel','water_heating'] * 100 / final_house_1.loc['bio_fuel','total']
w_heating_7 = final_house_1.loc['sum','water_heating'] * 100 / final_house_1.loc['sum','total']

cooking_1 = final_house_1.loc['electricity','cooking'] * 100 / final_house_1.loc['electricity','total']
cooking_3 = final_house_1.loc['gas','cooking'] * 100 / final_house_1.loc['gas','total']
cooking_4 = final_house_1.loc['solid_fuel','cooking'] * 100 / final_house_1.loc['solid_fuel','total']
cooking_5 = final_house_1.loc['oil','cooking'] * 100 / final_house_1.loc['oil','total']
cooking_7 = final_house_1.loc['sum','cooking'] * 100 / final_house_1.loc['sum','total']

lighting_1 = final_house_1.loc['electricity','lighting'] * 100 / final_house_1.loc['electricity','total']
lighting_7 = final_house_1.loc['sum','lighting'] * 100 / final_house_1.loc['sum','total']

other_1 = final_house_1.loc['electricity','other'] * 100 / final_house_1.loc['electricity','total']
other_7 = final_house_1.loc['sum','other'] * 100 / final_house_1.loc['sum','total']

In [120]:
final_house_1.loc['electricity','heating'] = heating_1
final_house_1.loc['gas','heating'] = heating_3
final_house_1.loc['solid_fuel','heating'] = heating_4
final_house_1.loc['oil','heating'] = heating_5
final_house_1.loc['bio_fuel','heating'] = heating_6
final_house_1.loc['sum','heating'] = heating_7

final_house_1.loc['electricity','conditioning'] = conditioning_1
final_house_1.loc['sum','conditioning'] = conditioning_7

final_house_1.loc['electricity','water_heating'] = w_heating_1
final_house_1.loc['gas','water_heating'] = w_heating_3
final_house_1.loc['solid_fuel','water_heating'] = w_heating_4
final_house_1.loc['oil','water_heating'] = w_heating_5
final_house_1.loc['bio_fuel','water_heating'] = w_heating_6
final_house_1.loc['sum','water_heating'] = w_heating_7

final_house_1.loc['electricity','cooking'] = cooking_1
final_house_1.loc['gas','cooking'] = cooking_3
final_house_1.loc['solid_fuel','cooking'] = cooking_4
final_house_1.loc['oil','cooking'] = cooking_5
final_house_1.loc['sum','cooking'] = cooking_7

final_house_1.loc['electricity','lighting'] = lighting_1
final_house_1.loc['sum','lighting'] = lighting_7

final_house_1.loc['electricity','other'] = other_1
final_house_1.loc['sum','other'] = other_7

In [121]:
final_house_1['total'] = [100, 0, 100, 100, 100, 100, 100]

In [122]:
final_house_1 = final_house_1.round({'heating':2, 'conditioning':2, 'water_heating':2, 'cooking':2, 'lighting':2,
                                     'other': 2, 'total': 2})

In [123]:
# final_house_1

In [124]:
new_category_10 = new_category_6.copy(deep = True)

In [125]:
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'el_heating'] = 0
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'el_conditioning'] = 0
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'el_water_heating'] = 0
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'el_cooking'] = 0
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'el_lighting'] = 0
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'el_other'] = 0
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'th_heating'] = 0
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'th_water_heating'] = 0
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'gas_heating'] = 0
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'gas_water_heating'] = 0
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'gas_cooking'] = 0
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'solid_heating'] = 0
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'solid_water_heating'] = 0
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'solid_cooking'] = 0
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'oil_heating'] = 0
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'oil_water_heating'] = 0
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'oil_cooking'] = 0
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'bio_heating'] = 0
new_category_10.loc[((new_category_10.house_type == 2) | (new_category_10.house_type == 3) | \
                    (new_category_10.house_type == 11) | (new_category_10.house_type == 22) | \
                    ((new_category_10.house_type == 1) & (new_category_10.heating == 2))),'bio_water_heating'] = 0

In [126]:
el_heating_sum = sum(new_category_10['el_heating'].values[:-1])
th_heating_sum = sum(new_category_10['th_heating'].values[:-1])
gas_heating_sum = sum(new_category_10['gas_heating'].values[:-1])
solid_heating_sum = sum(new_category_10['solid_heating'].values[:-1])
oil_heating_sum = sum(new_category_10['oil_heating'].values[:-1])
bio_heating_sum = sum(new_category_10['bio_heating'].values[:-1])

el_conditioning_sum = sum(new_category_10['el_conditioning'].values[:-1])

el_water_heating_sum = sum(new_category_10['el_water_heating'].values[:-1])
th_water_heating_sum = sum(new_category_10['th_water_heating'].values[:-1])
gas_water_heating_sum = sum(new_category_10['gas_water_heating'].values[:-1])
solid_water_heating_sum = sum(new_category_10['solid_water_heating'].values[:-1])
oil_water_heating_sum = sum(new_category_10['oil_water_heating'].values[:-1])
bio_water_heating_sum = sum(new_category_10['bio_water_heating'].values[:-1])

el_cooking_sum = sum(new_category_10['el_cooking'].values[:-1])
gas_cooking_sum = sum(new_category_10['gas_cooking'].values[:-1])
solid_cooking_sum = sum(new_category_10['solid_cooking'].values[:-1])
oil_cooking_sum = sum(new_category_10['oil_cooking'].values[:-1])

el_lighting_sum = sum(new_category_10['el_lighting'].values[:-1])

el_other_sum = sum(new_category_10['el_other'].values[:-1])

In [127]:
final_table_6 = pd.DataFrame({'heating': [0, 0, 0, 0, 0, 0], 'conditioning': [0, 0, 0, 0, 0, 0], 
                              'water_heating': [0, 0, 0, 0, 0, 0], 'cooking': [0, 0, 0, 0, 0, 0],
                              'lighting': [0, 0, 0, 0, 0, 0], 'other': [0, 0, 0, 0, 0, 0]})

In [128]:
final_table_6.loc[0, 'heating'] = el_heating_sum
final_table_6.loc[1, 'heating'] = th_heating_sum
final_table_6.loc[2, 'heating'] = gas_heating_sum
final_table_6.loc[3, 'heating'] = solid_heating_sum
final_table_6.loc[4, 'heating'] = oil_heating_sum
final_table_6.loc[5, 'heating'] = bio_heating_sum

final_table_6.loc[0, 'conditioning'] = el_conditioning_sum

final_table_6.loc[0, 'water_heating'] = el_water_heating_sum
final_table_6.loc[1, 'water_heating'] = th_water_heating_sum
final_table_6.loc[2, 'water_heating'] = gas_water_heating_sum
final_table_6.loc[3, 'water_heating'] = solid_water_heating_sum
final_table_6.loc[4, 'water_heating'] = oil_water_heating_sum
final_table_6.loc[5, 'water_heating'] = bio_water_heating_sum

final_table_6.loc[0, 'cooking'] = el_cooking_sum
final_table_6.loc[2, 'cooking'] = gas_cooking_sum
final_table_6.loc[3, 'cooking'] = solid_cooking_sum
final_table_6.loc[4, 'cooking'] = oil_cooking_sum

final_table_6.loc[0, 'lighting'] = el_lighting_sum

final_table_6.loc[0, 'other'] = el_other_sum

In [129]:
final_table_6['total'] = final_table_6.heating + final_table_6.conditioning + final_table_6.water_heating + \
                         final_table_6.cooking + final_table_6.lighting + final_table_6.other

In [130]:
final_table_6['category'] = (['electricity', 'thermal_energy', 'gas', 'solid_fuel', 'oil', 'bio_fuel'])

final_table_6 = final_table_6.reset_index().set_index('category')

final_table_6 = final_table_6.drop(labels='index', axis=1)

In [131]:
final_table_6.loc['sum'] = [0, 0, 0, 0, 0, 0, 0]

final_table_6.loc['sum', 'heating'] = sum(final_table_6['heating'].values)
final_table_6.loc['sum', 'conditioning'] = sum(final_table_6['conditioning'].values)
final_table_6.loc['sum', 'water_heating'] = sum(final_table_6['water_heating'].values)
final_table_6.loc['sum', 'cooking'] = sum(final_table_6['cooking'].values)
final_table_6.loc['sum', 'lighting'] = sum(final_table_6['lighting'].values)
final_table_6.loc['sum', 'other'] = sum(final_table_6['other'].values)
final_table_6.loc['sum', 'total'] = sum(final_table_6['total'].values)

In [132]:
final_flat = final_table_6.copy(deep = True)

In [133]:
# final_flat

,heating,conditioning,water_heating,cooking,lighting,other,total
category,,,,,,,
electricity,0.000,44.722,264.177,153.855,1098.873,4.26,1565.887
thermal_energy,1508.046,0.000,631.853,0.000,0.000,0.00,2139.899
gas,0.000,0.000,682.338,1595.866,0.000,0.00,2278.204
solid_fuel,0.000,0.000,0.000,0.000,0.000,0.00,0.000
oil,0.000,0.000,4.681,36.840,0.000,0.00,41.521
bio_fuel,0.000,0.000,0.000,0.000,0.000,0.00,0.000
sum,1508.046,44.722,1583.049,1786.561,1098.873,4.26,6025.511


In [134]:
final_flat_1 = final_flat.round({'heating':2, 'conditioning':2, 'water_heating':2, 'cooking':2, 'lighting':2,
                                 'other': 2, 'total': 2})

In [135]:
heating_1 = final_flat_1.loc['electricity','heating'] * 100 / final_flat_1.loc['electricity','total']
heating_2 = final_flat_1.loc['thermal_energy','heating'] * 100 / final_flat_1.loc['thermal_energy','total']
heating_3 = final_flat_1.loc['gas','heating'] * 100 / final_flat_1.loc['gas','total']
heating_5 = final_flat_1.loc['oil','heating'] * 100 / final_flat_1.loc['oil','total']
heating_7 = final_flat_1.loc['sum','heating'] * 100 / final_flat_1.loc['sum','total']

conditioning_1 = final_flat_1.loc['electricity','conditioning'] * 100 / final_flat_1.loc['electricity','total']
conditioning_7 = final_flat_1.loc['sum','conditioning'] * 100 / final_flat_1.loc['sum','total']

w_heating_1 = final_flat_1.loc['electricity','water_heating'] * 100 / final_flat_1.loc['electricity','total']
w_heating_2 = final_flat_1.loc['thermal_energy','water_heating'] * 100 / final_flat_1.loc['thermal_energy','total']
w_heating_3 = final_flat_1.loc['gas','water_heating'] * 100 / final_flat_1.loc['gas','total']
w_heating_5 = final_flat_1.loc['oil','water_heating'] * 100 / final_flat_1.loc['oil','total']
w_heating_7 = final_flat_1.loc['sum','water_heating'] * 100 / final_flat_1.loc['sum','total']

cooking_1 = final_flat_1.loc['electricity','cooking'] * 100 / final_flat_1.loc['electricity','total']
cooking_3 = final_flat_1.loc['gas','cooking'] * 100 / final_flat_1.loc['gas','total']
cooking_5 = final_flat_1.loc['oil','cooking'] * 100 / final_flat_1.loc['oil','total']
cooking_7 = final_flat_1.loc['sum','cooking'] * 100 / final_flat_1.loc['sum','total']

lighting_1 = final_flat_1.loc['electricity','lighting'] * 100 / final_flat_1.loc['electricity','total']
lighting_7 = final_flat_1.loc['sum','lighting'] * 100 / final_flat_1.loc['sum','total']

other_1 = final_flat_1.loc['electricity','other'] * 100 / final_flat_1.loc['electricity','total']
other_7 = final_flat_1.loc['sum','other'] * 100 / final_flat_1.loc['sum','total']

In [136]:
final_flat_1.loc['electricity','heating'] = heating_1
final_flat_1.loc['thermal_energy','heating'] = heating_2
final_flat_1.loc['gas','heating'] = heating_3
final_flat_1.loc['oil','heating'] = heating_5
final_flat_1.loc['sum','heating'] = heating_7

final_flat_1.loc['electricity','conditioning'] = conditioning_1
final_flat_1.loc['sum','conditioning'] = conditioning_7

final_flat_1.loc['electricity','water_heating'] = w_heating_1
final_flat_1.loc['thermal_energy','water_heating'] = w_heating_2
final_flat_1.loc['gas','water_heating'] = w_heating_3
final_flat_1.loc['oil','water_heating'] = w_heating_5
final_flat_1.loc['sum','water_heating'] = w_heating_7

final_flat_1.loc['electricity','cooking'] = cooking_1
final_flat_1.loc['gas','cooking'] = cooking_3
final_flat_1.loc['oil','cooking'] = cooking_5
final_flat_1.loc['sum','cooking'] = cooking_7

final_flat_1.loc['electricity','lighting'] = lighting_1
final_flat_1.loc['sum','lighting'] = lighting_7

final_flat_1.loc['electricity','other'] = other_1
final_flat_1.loc['sum','other'] = other_7

In [137]:
final_flat_1['total'] = [100, 100, 100, 0, 100, 0, 100]

In [138]:
final_flat_1 = final_flat_1.round({'heating':2, 'conditioning':2, 'water_heating':2, 'cooking':2, 'lighting':2,
                                   'other': 2, 'total': 2})

In [139]:
# final_flat_1

In [140]:
# final_flat_1.to_excel("flat_per.xlsx")
# final_house_1.to_excel("house_per.xlsx")

In [141]:
# final_flat.to_excel("flat.xlsx")
# final_house.to_excel("house.xlsx")

In [142]:
# new_category_6.to_csv("iter_approx.csv", sep = ";")
# new_category_6.to_excel("iter_approx.xlsx", index = False)